In [51]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as Data
import torchvision

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

use_cuda=False

def XnumpyToTensor(x_data_np):
    x_data_np = np.array(x_data_np, dtype=np.float32)        
    print(x_data_np.shape)
    print(type(x_data_np))

    if use_cuda:
        print("Using the GPU")    
        X_tensor = Variable(torch.from_numpy(x_data_np).cuda()) # Note the conversion for pytorch    
    else:
        print("Using the CPU")
        X_tensor = Variable(torch.from_numpy(x_data_np)) # Note the conversion for pytorch
    
    print(type(X_tensor.data)) # should be 'torch.cuda.FloatTensor'
    print(x_data_np.shape)
    print(type(x_data_np))    
    return X_tensor


# Convert the np arrays into the correct dimention and type
# Note that BCEloss requires Float in X as well as in y
def YnumpyToTensor(y_data_np):    
    y_data_np=y_data_np.reshape((y_data_np.shape[0],1)) # Must be reshaped for PyTorch!
    print(y_data_np.shape)
    print(type(y_data_np))

    if use_cuda:
        print("Using the GPU")            
    #     Y = Variable(torch.from_numpy(y_data_np).type(torch.LongTensor).cuda())
        Y_tensor = Variable(torch.from_numpy(y_data_np)).type(torch.FloatTensor).cuda()  # BCEloss requires Float        
    else:
        print("Using the CPU")        
    #     Y = Variable(torch.squeeze (torch.from_numpy(y_data_np).type(torch.LongTensor)))  #         
        Y_tensor = Variable(torch.from_numpy(y_data_np)).type(torch.FloatTensor)  # BCEloss requires Float        

    print(type(Y_tensor.data)) # should be 'torch.cuda.FloatTensor'
    print(y_data_np.shape)
    print(type(y_data_np))    
    return Y_tensor


In [52]:
path = 'data/mnist/'

In [53]:
raw_train = pd.read_csv(path + 'train.csv')
raw_test = pd.read_csv(path + 'train.csv')

In [54]:
raw_train_array = raw_train.values
raw_test_array = raw_test.values

In [55]:
raw_train_array = np.random.permutation(raw_train_array)
len(raw_train_array)

42000

In [56]:
raw_train = raw_train_array[:40000, :]

# raw_train=XnumpyToTensor (raw_train)

raw_valid = raw_train_array[40000:, :]

# raw_valid=XnumpyToTensor(raw_valid)


In [57]:
train_label = np.eye(10)[raw_train[:,0]]
train_data = raw_train[:,1:]
valid_label = np.eye(10)[raw_valid[:,0]]
valid_data = raw_valid[:,1:]

train_data.shape

(40000, 784)

In [58]:
def reshape(data, target_size): return np.reshape(data, target_size)

In [59]:
train_data = reshape(train_data, [40000, 1, 28, 28])
valid_data = reshape(valid_data, [2000, 1, 28, 28])
train_data.shape, train_label.shape, valid_label.shape, valid_data.shape

print(train_data.shape)
print(type(train_data))



(40000, 1, 28, 28)
<type 'numpy.ndarray'>


In [60]:
train_data = np.array(train_data, dtype=np.float32)  
valid_data = np.array(valid_data, dtype=np.float32)  

In [61]:
BATCH_SIZE = 64
LEARNING_RATE = 0.1
# EPOCH = 2

In [62]:
#convert to pytorch tensor
train_data = torch.from_numpy(train_data)
train_label = torch.from_numpy(train_label)
val_data = torch.from_numpy(valid_data)
val_label = torch.from_numpy(valid_label)

train_data.size(),train_label.size(),val_data.size(),val_label.size()

(torch.Size([40000, 1, 28, 28]),
 torch.Size([40000, 10]),
 torch.Size([2000, 1, 28, 28]),
 torch.Size([2000, 10]))

In [63]:
train_data.size(0), train_label.size(0)

(40000, 40000)

In [64]:
train_dataset = Data.TensorDataset(data_tensor=train_data, target_tensor=train_label)
val_dataset = Data.TensorDataset(data_tensor=val_data, target_tensor=val_label)
train_loader = Data.DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
val_loader = Data.DataLoader(dataset=val_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)

In [70]:
#pyton opp
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        #in_chanel out_chanel kernel stride padding
        self.conv1 = nn.Conv2d(1, 32, 3)
        self.conv2 = nn.Conv2d(32, 32, 3)
        self.conv3 = nn.Conv2d(32, 64, 3)
        self.conv4 = nn.Conv2d(64, 64, 3)
        
        self.fc1 = nn.Linear(64*4*4, 512)
        self.fc2 = nn.Linear(512, 10)
        
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        
        x = F.relu(self.conv3(x))        
        x = F.max_pool2d(F.relu(self.conv4(x)), 2) 
        
        x = x.view(x.size(0), -1)
#         x = F.relu(F.batch_norm(self.fc1(x)))
        x = F.softmax(F.dropout(F.batch_norm(self.fc2(x)), .5))
        
    def num_flat_features(self, x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
    
# CNN = torch.nn.Sequential(
#     nn.Conv2d(1, 32, 3),
#     nn.Conv2d(1, 32, 3),
#     nn.ReLU(),
#     nn.MaxPool2d(2),
#     nn.Conv2d(1, 32, 3),
#     nn.Conv2d(1, 32, 3),
#     nn.ReLU(),
#     nn.MaxPool2d(2),
#     nn.Linear(64*4*4, 3)
#     nn.Linear(521, 10)
# )
        
cnn = CNN()
print(cnn)

CNN (
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (conv4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear (1024 -> 512)
  (fc2): Linear (512 -> 10)
)


In [71]:
list(cnn.parameters())[2].size() #conv2 weights

torch.Size([32, 32, 3, 3])

In [72]:
#Loss and Optimizer

In [73]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(cnn.parameters(), lr=LEARNING_RATE)

In [74]:
#train the model
for epoch in range(2):
    for i, (images, labels) in enumerate(train_loader):
        print(type(images))
        images = Variable(images)
        labels = Variable(labels)
        print(type(images))
        # Forward + Backward + Optimize
        optimizer.zero_grad()
        outputs = cnn(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print(loss.data)
            print ('Epoch [%d/%d], Iter [%d/%d] Loss: %.4f' 
                   %(epoch+1, 2, i+1, len(train_dataset)//BATCH_SIZE, loss.data[0]))
        

<class 'torch.FloatTensor'>
<class 'torch.autograd.variable.Variable'>


RuntimeError: size mismatch, m1: [64 x 1024], m2: [512 x 10] at /home/gpu/dev/pytorch/torch/lib/TH/generic/THTensorMath.c:1293

In [60]:
# save and load(保存和提取)

In [63]:
# save
def save():
    pass
    #torch.save(net_name, 'net.pkl')
    #torch.save(net_name.state_dict(), 'net_params.pkl')
# load
def restore_net():
    pass
    #net_new = torch.load('net.pkl')

def restore_params():
    pass
    #net_new_old_params = NET()
    #net_new_old_params = net_new_old_params.load_state_dict(torch.load()'net_params.pkl'))


In [65]:
#批处理

In [72]:
#optimizer 优化器
# optimizer = torch.optim.SGD()
# torch.optim.Adam
# momentum (m)
# alpha (RMSprop)
# Adam (betas)